# Projeto de Destoxificação com Sentence Transformers
Este notebook carrega embeddings de texto previamente salvos, calcula centróides por classe de toxicidade, e aplica transformações vetoriais para encontrar versões destoxificadas dos textos.

In [1]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Carregar o CSV com embeddings salvos
df = pd.read_csv("../data/train_embedded.csv")

# Converter coluna de string para vetor
df["embedding"] = df["embedding"].apply(ast.literal_eval)

In [2]:
# Definir rótulos de toxicidade
toxic_labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Filtrar subset não tóxico
non_toxic_subset = df[df[toxic_labels].sum(axis=1) == 0]

In [3]:
# Calcular vetores médios por classe tóxica
centroids = {}

for label in toxic_labels:
    subset = df[df[label] == 1.0]
    if not subset.empty:
        embeddings = np.vstack(subset["embedding"].values)
        centroids[label] = np.mean(embeddings, axis=0)

# Calcular centroide de textos não tóxicos
non_toxic_embeddings = np.vstack(non_toxic_subset["embedding"].values)
centroids["non_toxic"] = np.mean(non_toxic_embeddings, axis=0)

In [14]:
# Selecionar um comentário tóxico
example_row = df[df["obscene"] == 1.0].iloc[0]
v_tox = example_row["embedding"]
original_text = example_row["comment_text"]

# Aplicar deslocamento vetorial
v_med_tox = centroids["obscene"]
v_med_nontox = centroids["non_toxic"]
v_destox = v_tox + (v_med_nontox - v_med_tox)

In [12]:
# Preparar vetores e textos não tóxicos
non_toxic_texts = non_toxic_subset["comment_text"].tolist()
non_toxic_vectors = np.vstack(non_toxic_subset["embedding"].values)

# Calcular similaridade e buscar o mais próximo
sims = cosine_similarity([v_destox], non_toxic_vectors)[0]
top_idx = np.argmax(sims)

# Exibir resultados
print("🧪 Texto original (tóxico):")
print(original_text)

print("\n🔄 Texto mais próximo do vetor destoxificado:")
print(non_toxic_texts[top_idx])

🧪 Texto original (tóxico):
cock  suck before you piss around on my work

🔄 Texto mais próximo do vetor destoxificado:
dont care  leave me alone  shit head  you have better things to do and let me do my work alone 
